In [1]:
import sys
import os

sys.path.append(os.path.join(
    r'C:\Users\AA2-PC2\Software\NI-experiment-control - Atom Array\nistreamer'
))

In [2]:
from nistreamer.streamer import NIStreamer
from nistreamer.utils import iplot, RendOption

### Setup cards

In [3]:
nistrm = NIStreamer()

In [4]:
TRIG_LINE = 'RTSI0'
REF_CLK_LINE = 'RTSI7'

In [5]:
ao_card = nistrm.add_ao_card('Dev2', samp_rate=1e6)

ao_card.cfg_start_trig(line=TRIG_LINE, export=True)
ao_card.cfg_ref_clk(line=REF_CLK_LINE, export=True)

In [6]:
mf_card = nistrm.add_ao_card('Dev3', samp_rate=1e6)

mf_card.cfg_start_trig(line=TRIG_LINE)
mf_card.cfg_ref_clk(line=REF_CLK_LINE)

In [7]:
ao_card.add_chan(chan_idx=0)
mf_card.add_chan(chan_idx=0)

Channel ao0 on card Dev3

In [ ]:
nistrm.reset_all()

### Closing edge / keep value for final pulse

#### Finite-duration pulse

In [15]:
# Closing edge forming - const pulse

nistrm.clear_edit_cache()

ao_card['ao0'].constant(t=0.0, dur=1e-3, val=1.0)
mf_card['ao0'].constant(t=0.0, dur=1e-3, val=1.0)

print(nistrm.compile())
nistrm.stream_exp()

0.001001


In [50]:
# Closing edge forming - sine pulse

import numpy as np

nistrm.clear_edit_cache()

ao_card['ao0'].sine(t=0.0, dur=1e-3, amp=1.0, freq=10e3, phase=0.5*np.pi)
mf_card['ao0'].sine(t=0.0, dur=1e-3, amp=1.0, freq=10e3, phase=0.5*np.pi)

print(nistrm.compile())
nistrm.stream_exp()

0.001001


In [51]:
# keep_val = True

import numpy as np

nistrm.clear_edit_cache()

ao_card['ao0'].sine(t=0.0, dur=1e-3, amp=1.0, freq=10e3, phase=0.5*np.pi, keep_val=True)
mf_card['ao0'].sine(t=0.0, dur=1e-3, amp=1.0, freq=10e3, phase=0.5*np.pi, keep_val=True)

print(nistrm.compile())
nistrm.stream_exp()

0.001001


In [70]:
# stop_time=2e-3 - should give precise total run time of 2 ms

import numpy as np

nistrm.clear_edit_cache()

ao_card['ao0'].sine(t=0.0, dur=1e-3, amp=1.0, freq=10e3, phase=0.5*np.pi, keep_val=True)
mf_card['ao0'].sine(t=0.0, dur=1e-3, amp=1.0, freq=10e3, phase=0.5*np.pi, keep_val=True)

print(nistrm.compile(2e-3))
nistrm.stream_exp()

0.002


#### "Go-something" pulse

In [104]:
nistrm.reset_all()

In [89]:
# To avoid 1-sample error (min for NI cards is 2), either start pulse at 1us
nistrm.clear_edit_cache()

ao_card['ao0'].go_constant(t=1e-6, val=1.0)
mf_card['ao0'].go_constant(t=1e-6, val=1.0)

print(nistrm.compile())
nistrm.stream_exp()

2e-06


In [86]:
# To avoid 1-sample error (min for NI cards is 2), or compile with stop_time = 2us
nistrm.clear_edit_cache()

ao_card['ao0'].go_constant(t=0, val=1.0)
mf_card['ao0'].go_constant(t=0, val=1.0)

print(nistrm.compile(2e-6))
nistrm.stream_exp()

2e-06


In [103]:
nistrm.clear_edit_cache()

ao_card['ao0'].go_sine(t=0, amp=1.0, freq=10e3, phase=0.12345*np.pi)
mf_card['ao0'].go_sine(t=0, amp=1.0, freq=10e3, phase=0.12345*np.pi)

print(nistrm.compile(1e-3))
nistrm.stream_exp()

0.001


### `add_reset_instr()`

In [123]:
nistrm.clear_edit_cache()

ao_card['ao0'].sine(t=1.0e-3, dur=1.0e-3, amp=1.0, freq=1.2e3, keep_val=True)  # constant(t=1.0e-3, dur=1.0e-3, val=1.0)
mf_card['ao0'].go_constant(t=0, val=1.0)
nistrm.add_reset_instr()

print(nistrm.compile())
nistrm.stream_exp()

0.0020009999999999997


In [124]:
nistrm.clear_edit_cache()

ao_card['ao0'].sine(t=1.0e-3, dur=1.0e-3, amp=1.0, freq=1.2e3, keep_val=True)
mf_card['ao0'].go_constant(t=0, val=1.0)
nistrm.add_reset_instr(3e-3)

print(nistrm.compile())
nistrm.stream_exp()

0.0030009999999999998


In [121]:
nistrm.clear_edit_cache()

ao_card['ao0'].go_sine(t=1.0e-3, amp=1.0, freq=1.2e3)
mf_card['ao0'].go_constant(t=0, val=1.0)
nistrm.add_reset_instr(3e-3)

print(nistrm.compile())
nistrm.stream_exp()

0.0030009999999999998
